# Traitement des données de financements 

In [3]:
#importer les packages
import pandas as pd
import numpy as np
from project.server.main.utils import replace_all,get_scanR_structure,get_id
from project.server.main.cached_data_handler import get_structure, get_person
from project.server.main.features_into_dictionnary import persons, projects, address
from tqdm import tqdm
import pprint as pp
tqdm.pandas()
from project.server.main.pickle import load_cache,write_cache
import os
from dotenv import load_dotenv
import requests

load_dotenv()

Authorization = os.getenv('Authorization_access_185')
Authorization_ORCID = os.getenv('Authorization_cluster_BSO_ORCID')
url_cluster = os.getenv('url_cluster')

In [4]:
sources=pd.read_json('sources.json')

In [ ]:
source=list(sources.keys())[1]
source

In [ ]:
#cache structures, personnes et orcid avec differentes sources de donnees
cached_data = {}
try:
    cached_data = load_cache(cached_data,f"./DATA/{source}/caches/cached_{source.lower()}_data.pkl")
except:
    write_cache(cached_data,f"./DATA/{source}/caches/cached_{source.lower()}_data.pkl")
    
cached_data_persons = {}
try:
    cached_data_persons = load_cache(cached_data_persons,f"./DATA/{source}/caches/cached_{source.lower()}_data_persons.pkl")
except:
    write_cache(cached_data_persons,f"./DATA/{source}/caches/cached_{source.lower()}_data_persons.pkl")
    
cached_data_orcid = {}
try:
    cached_data_orcid = load_cache(cached_data_orcid,f"./DATA/{source}/caches/cached_{source.lower()}_data_orcid.pkl")
except:
    write_cache(cached_data_orcid,f"./DATA/{source}/caches/cached_{source.lower()}_data_orcid.pkl")

# Données partenaires

In [21]:
# amener les partenaires depuis le site 
if source=='ANR':
    page_partenaires_10 = requests.get(sources[source]['url_partners']).json()
    colonnes_partenaires_10 = page_partenaires_10['columns']
    donnees_partenaires_10 = page_partenaires_10['data']
    df_partenaires=pd.DataFrame(data=donnees_partenaires_10,columns=colonnes_partenaires_10)
elif source=='ANSES':
    df_from_anses=pd.read_excel(sources[source]['url_partners'])
    df=df_from_anses.iloc[1:,:]
    df.columns=list(df_from_anses.iloc[0,:])
    dict_equipe={list(df_from_anses.columns)[k].replace('Équipe 10 ','Équipe 10').replace('Équipe13','Équipe 13'):k for k in range (len(list(df_from_anses.columns))) if list(df_from_anses.columns)[k].find('Équipe')>=0}
    list_df=[]
    number=3
    for n in range(1,len(dict_equipe)+1):
        equipe_n=pd.concat([df.iloc[:,0:3],df.iloc[:,number:number+6]], axis=1)
        list_df.append(equipe_n)
        number+=6
    df_partenaires=pd.concat([list_df[k].dropna(subset=[sources[source]['nom'], sources[source]['prenom'],sources[source]['nom_structure'], sources[source]['nom'], 'Pays'], how='all') for k in range(len(list_df))])
    
elif source=='IRESP':
    df_partenaires1=pd.read_csv(sources[source]['url_partners1'] ,sep=";", encoding='UTF-8')
    df_partenaires2=pd.read_csv(sources[source]['url_partners2'] ,sep=";", encoding='UTF-8')
    df_partenaires=pd.concat([df_partenaires1,df_partenaires2])
elif source=='ADEME':
    df_partenaires=pd.read_csv(sources[source]['url_partners'] ,sep=",", encoding='ISO-8859-1', on_bad_lines='skip')
else:    
    df_partenaires=pd.read_csv(sources[source]['url_partners'] ,sep=";", encoding='ISO-8859-1')

df_partenaires=df_partenaires.reset_index()
del df_partenaires['index']

# Matcher établissement

In [22]:
id_struct=df_partenaires
id_struct[f"{sources[source]['nom_structure']}2"]=id_struct.loc[:,sources[source]['nom_structure']].apply(lambda x: replace_all(str(x).lower().replace(" d e"," d'e").replace(" d a"," d'a").replace(" d i"," d'i").replace(" d o"," d'o").replace(" d u"," d'u").replace(" d y"," d'y").replace(" d h"," d'h").replace(" l e"," l'e").replace(" l a"," l'a").replace(" l i"," l'i").replace(" l o"," l'o").replace(" l u"," l'u").replace(" l y"," l'y").replace(" l h"," l'h")))
id_struct=id_struct.drop_duplicates(subset=[f"{sources[source]['nom_structure']}2"])

In [ ]:
id_struct.progress_apply(lambda row: get_structure(row,source,cached_data,sources[source]['nom_structure'],sources[source]['ville'],sources[source]['pays'],sources[source]['code_projet'],False), axis=1) 
write_cache(cached_data,f"./DATA/{source}/caches/cached_{source}_data.pkl")

In [ ]:
len(cached_data)

In [ ]:
id_struct['id_structure_matcher']=id_struct.loc[:,sources[source]['nom_structure']].apply(lambda x: cached_data[x])

In [ ]:
id_struct

In [27]:
id_struct=id_struct.reset_index()
del id_struct['index']
id_struct.to_excel(f"./DATA/{source}/df_partenaires.xlsx")
id_struct.to_json(f"./DATA/{source}/df_partenaires.json")

In [ ]:
id_struct=pd.read_json(f"./DATA/{source}/df_partenaires.json")
id_struct=id_struct[[sources[source]['nom_structure'],'id_structure_matcher']]
id_struct[f"{sources[source]['nom_structure']}2"]=id_struct.loc[:,sources[source]['nom_structure']].apply(lambda x: replace_all(str(x).lower().replace(" d e"," d'e").replace(" d a"," d'a").replace(" d i"," d'i").replace(" d o"," d'o").replace(" d u"," d'u").replace(" d y"," d'y").replace(" d h"," d'h").replace(" l e"," l'e").replace(" l a"," l'a").replace(" l i"," l'i").replace(" l o"," l'o").replace(" l u"," l'u").replace(" l y"," l'y").replace(" l h"," l'h")))

df_partenaires[f"{sources[source]['nom_structure']}2"]=df_partenaires.loc[:,sources[source]['nom_structure']].apply(lambda x: replace_all(str(x).lower().replace(" d e"," d'e").replace(" d a"," d'a").replace(" d i"," d'i").replace(" d o"," d'o").replace(" d u"," d'u").replace(" d y"," d'y").replace(" d h"," d'h").replace(" l e"," l'e").replace(" l a"," l'a").replace(" l i"," l'i").replace(" l o"," l'o").replace(" l u"," l'u").replace(" l y"," l'y").replace(" l h"," l'h")))
df_partenaires_struct=pd.merge(df_partenaires,id_struct[[f"{sources[source]['nom_structure']}2",'id_structure_matcher']], on=f"{sources[source]['nom_structure']}2", how='left')
df_partenaires_struct

In [ ]:
#compléter les données avec scanR
url_scanr='https://storage.gra.cloud.ovh.net/v1/AUTH_32c5d10cb0fe4519b957064a111717e3/scanR/projects.json'
requete_scanR = requests.get(url_scanr)
page_scanR= requete_scanR.json()
df_scanR=pd.DataFrame(page_scanR)
scanR=df_scanR.explode('participants').loc[:,['id','participants']]
scanR=scanR.rename(columns={'id':'id_anr'})
scanR['index']=[x for x in range(len(scanR))]
scanR=scanR.set_index('index')
scanR['id_structure_scanr']=scanR['participants'].apply(lambda x: x.get(str('structure')) if isinstance(x, dict) else None )
scanR['nom_struct']=scanR['participants'].apply(lambda x: get_scanR_structure(x))
del scanR['participants']
scanR_nettoye=scanR.drop_duplicates(subset='nom_struct')
scanR_nettoye[f"{sources[source]['nom_structure']}2"]=scanR_nettoye.loc[:,'nom_struct'].apply(lambda x: replace_all(str(x).lower()))
scanR_nettoye=scanR_nettoye[['id_structure_scanr',f"{sources[source]['nom_structure']}2"]]
scanR_nettoye=scanR_nettoye.drop_duplicates(subset=f"{sources[source]['nom_structure']}2")

In [ ]:
df_partenaires_struct=pd.merge(df_partenaires_struct,scanR_nettoye, on=f"{sources[source]['nom_structure']}2", how='left')
df_partenaires_struct

In [ ]:
#######fichier avec les identifiants structures rettrouvés à la main par Emmanuel ==> 'code'
scanr_structures=pd.read_excel('scanr_partenaires_non_identifies.xlsx')
scanr_structures[f"{sources[source]['nom_structure']}2"]=scanr_structures.loc[:,'Nom'].apply(lambda x: replace_all(str(x).lower().replace(" d e"," d'e").replace(" d a"," d'a").replace(" d i"," d'i").replace(" d o"," d'o").replace(" d u"," d'u").replace(" d y"," d'y").replace(" d h"," d'h").replace(" l e"," l'e").replace(" l a"," l'a").replace(" l i"," l'i").replace(" l o"," l'o").replace(" l u"," l'u").replace(" l y"," l'y").replace(" l h"," l'h")))
scanr_structures=scanr_structures[[f"{sources[source]['nom_structure']}2",'code']]
scanr_structures=scanr_structures.dropna().drop_duplicates(subset=f"{sources[source]['nom_structure']}2")
df_partenaires_complet=pd.merge(df_partenaires_struct,scanr_structures, on=f"{sources[source]['nom_structure']}2", how='left')
df_partenaires_complet

In [32]:
if 'finess' in list(df_partenaires.columns):
    finess_siret=pd.read_json("finess_siret-siege.json")
    df_partenaires_complet=pd.merge(df_partenaires_complet,finess_siret,how='left', on='finess')

In [ ]:
df_partenaires_complet[sources[source]['identifiants_preferes_structure']]

In [ ]:
df_partenaires_complet.columns

In [ ]:
df_partenaires_complet['id_structure']=df_partenaires_complet.apply(lambda row: get_id(row,sources[source]['identifiants_preferes_structure']), axis=1)
df_partenaires_complet

In [ ]:
#df_partenaires_complet[pd.isna(df_partenaires_complet.id_structure)]
df_partenaires_complet.loc[(pd.isna(df_partenaires_complet['id_structure']))|(str(df_partenaires_complet['id_structure'])=='None')|(str(df_partenaires_complet['id_structure'])=='nan')]

In [38]:
#df_partenaires_complet.to_excel(f"./DATA/{source}/df_partenaires_id_structures.xlsx")
df_partenaires_complet.to_json(f"./DATA/{source}/df_partners_id_structures.json")

In [22]:
df_partenaires_complet=pd.read_json(f"./DATA/{source}/df_partenaires_id_structures.json")

In [23]:
########récupération des structures sans identifiants pour les donner à Emmanuel
identifiants_a_remplir=df_partenaires_complet.loc[(pd.isna(df_partenaires_complet['id_structure']))|(str(df_partenaires_complet['id_structure'])=='None')|(str(df_partenaires_complet['id_structure'])=='nan')]
identifiants_a_remplir
identifiants_a_remplir=identifiants_a_remplir.drop_duplicates(subset=f"{sources[source]['nom_structure']}2")
identifiants_a_remplir=identifiants_a_remplir.reset_index()
del identifiants_a_remplir['index']

In [25]:
if sources[source]['ville'] in list(identifiants_a_remplir.columns) and sources[source]['pays'] in list(identifiants_a_remplir.columns) and sources[source]['adresse'] not in list(identifiants_a_remplir.columns):
    identifiants_a_remplir=identifiants_a_remplir[[sources[source]['nom_structure'],sources[source]['ville'],sources[source]['pays']]]
elif sources[source]['ville'] in list(identifiants_a_remplir.columns) and sources[source]['pays'] in list(identifiants_a_remplir.columns) and sources[source]['adresse'] in list(identifiants_a_remplir.columns):
    identifiants_a_remplir=identifiants_a_remplir[[sources[source]['nom_structure'],sources[source]['adresse'],sources[source]['ville'],sources[source]['pays']]]
elif sources[source]['region'] in list(identifiants_a_remplir.columns):
    identifiants_a_remplir=identifiants_a_remplir[[sources[source]['nom_structure'],sources[source]['region']]]
elif sources[source]['ville'] in list(identifiants_a_remplir.columns) and sources[source]['pays'] not in list(identifiants_a_remplir.columns):
    identifiants_a_remplir=identifiants_a_remplir[[sources[source]['nom_structure'],sources[source]['ville']]]
identifiants_a_remplir

In [26]:
identifiants_a_remplir.to_excel(f"./structures_manquantes/partenaires_non_identifies_{source}.xlsx", index=False)

# Matcher des chercheurs

In [ ]:
df_partenaires=pd.read_json(f"./DATA/{source}/df_partners_id_structures.json")

In [ ]:
if len([x for x in ['nom', 'prenom'] if x in list(sources[source].keys())])==2:
    df_partenaires['id_personne']=df_partenaires.progress_apply(lambda row: get_person(row, cached_data_persons,sources[source]['nom'],sources[source]['prenom']), axis=1)
    write_cache(cached_data_persons,f"./DATA/{source}/caches/cached_data_persons.pkl")
    #df_partenaires.to_excel(f"./DATA/{source}/df_partners_id_personne.xlsx")
    df_partenaires.to_json(f"./DATA/{source}/df_partners_id_personne.json")

In [ ]:
df_partenaires['id_personne']=df_partenaires.progress_apply(lambda row: get_person(row, cached_data_persons,sources[source]['nom'],sources[source]['prenom']), axis=1)

In [ ]:
df_partenaires.columns

In [ ]:
if sources[source]['id_ORCID'] in list(df_partenaires.columns):
    df_partenaires=pd.read_json(f"./DATA/{source}/df_partenaires_id_personne.json")
    df_partenaires['idref_ORCID']=df_partenaires.progress_apply(lambda row: orcid_to_idref(row,cached_data_orcid,sources[source]['id_ORCID'],Authorization_ORCID), axis=1)
    write_cache(cached_data_orcid,f"./DATA/{source}/caches/cached_data_orcid.pkl")
    df_partenaires.to_excel(f"./DATA/{source}/df_partenaires_id_personne_ORCID.xlsx")
    df_partenaires.to_json(f"./DATA/{source}/df_partenaires_id_personne_ORCID.json")

# ENVOI DES PROJETS SUR SCANR

In [41]:
if len(sources[source]['identifiants_preferes_personne'])==2:
    df_partenaires=pd.read_json(f"./DATA/{source}/df_partners_id_person_ORCID.json")
elif len(sources[source]['identifiants_preferes_personne'])==1:
    df_partenaires=pd.read_json(f"./DATA/{source}/df_partners_id_person.json")
else:
    df_partenaires=pd.read_json(f"./DATA/{source}/df_partners_id_structures.json")

In [42]:
df_partenaires.loc[df_partenaires.id_structure.apply(lambda x :isinstance(x,list)),'id_structure']=df_partenaires.loc[df_partenaires.id_structure.apply(lambda x :isinstance(x,list)),'id_structure'].apply(lambda y: y[0])

In [ ]:
if len([x for x in ['nom', 'prenom'] if x in list(sources[source].keys())])==2:
    df_partenaires['id_person']=df_partenaires.apply(lambda row: get_id(row,sources[source]['identifiants_preferes_personne']), axis=1)
    df_partenaires['persons']=df_partenaires.progress_apply(lambda row: persons(row,sources[source]['prenom'],sources[source]['nom']) ,axis=1)
else:
    df_partenaires['persons']=np.nan

In [44]:
if source != 'SIRANO':
    df_partenaires=df_partenaires.groupby([sources[source]['code_projet']]).agg({'persons': lambda x: [ y for y in x.tolist() if pd.isna(y)==False]}, dropna=False).reset_index()
else:
    df_projets=df_partenaires.groupby([sources[source]['code_projet'], sources[source]['annee'], sources[source]['acronyme'],sources[source]['titre'],sources[source]['budget']], dropna=False).agg({'persons': lambda x: [ y for y in x.tolist() if pd.isna(y)==False]}, dropna=False)

In [45]:
# amener les projets depuis le site 
if source=='ANR':
    page_projets_10 = requests.get(sources[source]['url_projets']).json()
    colonnes_projets_10 = page_projets_10['columns']
    donnees_projets_10 = page_projets_10['data']
    df_projets=pd.DataFrame(data=donnees_projets_10,columns=colonnes_projets_10)
elif source=='IRESP':
    df_projets1=pd.read_csv(sources[source]['url_projets1'] ,sep=";", encoding='UTF-8')
    df_projets2=pd.read_csv(sources[source]['url_projets2'] ,sep=";", encoding='UTF-8')
    df_projets=pd.concat([df_projets1,df_projets2])
    df_projets.loc[pd.isna(df_projets['Titre_du_projet_FR']),'Titre_du_projet_FR']=df_projets.loc[pd.isna(df_projets['Titre_du_projet_FR']),'Titre_du_projet']
elif source!='SIRANO':
    df_projets=pd.read_csv(sources[source]['url_projects'] ,sep=";", encoding='ISO-8859-1')

df_projets=df_projets.reset_index()
del df_projets['index']

In [ ]:
if source!='SIRANO':
    df_projets=pd.merge(df_projets,df_partenaires,on=sources[source]['code_projet'], how='left')
else :
    df_projets['id']=df_partenaires.apply(lambda row: f"{row[sources[source]['code_projet']]}-{row[sources[source]['annee']]}-{row[sources[source]['acronyme']]}" , axis=1)
    del df_projets['code_projet']
    sources[source]['code_projet']='id'

df_projets['type']=source
df_projets['name']=df_projets.progress_apply(lambda row: projects(row,sources[source]['titre_fr'],sources[source]['titre_en']) ,axis=1)
df_projets['description']=df_projets.progress_apply(lambda row: projects(row,sources[source]['resume_fr'],sources[source]['resume_en']) ,axis=1)
df_projets.loc[:,sources[source]['budget']]=df_projets.loc[:,sources[source]['budget']].apply(lambda x : float(str(x).replace('.0','').replace('.00','').replace(' ','').replace(',','.').replace('€','').replace('\x80','')))
df_projets=df_projets.rename(columns={sources[source]['annee']:'year',sources[source]['acronyme']:'acronym',
                                      sources[source]['budget']:'budget_financed',sources[source]['code_projet']:'id'})
df_projets=df_projets[['id','type','name','description','acronym','year','budget_financed','persons']]

In [ ]:
dict_row=df_projets.iloc[0,:].to_dict()
dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x')&(str(v)!='[]'))}
dict_row2

In [ ]:
#envoi
err=[]
for i,row in df_projets.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x')&(str(v)!='[]'))}
    try:
       r=requests.post('http://185.161.45.213/projects/projects', json = dict_row2, headers={"Authorization":Authorization})
       res= r.json()
       if res.get('status')=='ERR':
           err.append(res)
           if res.get('error').get('code')!=422:
               print(err)
               pp.pprint(err)
    except Exception as e:
        pp.pprint(e)

In [ ]:
pd.Series([x.get('issues').get('id') for x in err]).drop_duplicates().tolist() 

pour mettre à jour

In [ ]:
nbr_page=int(requests.get('http://185.161.45.213/projects/projects?where={"type":"ANSES"}&projection={"id":1}&max_results=500&page=1', headers={"Authorization":Authorization}).json()['hrefs']['last']['href'].split('page=')[1])

list_ids=[]
for i in range(1,nbr_page+1):
    print("page",i)
    page=requests.get('http://185.161.45.213/projects/projects?where={"type":"ANSES"}&projection={"id":1}&max_results=500'+f"&page={i}", headers={"Authorization":Authorization}).json()
    for k in range(len(page['data'])):
        print("k",k)
        list_ids.append(page['data'][k]['id'])
    
projets_a_ajouter=[x for x in list(df_projets['id']) if x not in list_ids]

projets_a_retirer=[x for x in list_ids if x not in list(df_projets['id'])]

df_projets = df_projets[df_projets['id'].apply(lambda x: x in projets_a_ajouter)]

In [ ]:
len(projets_a_ajouter)

In [ ]:
len(df_projets[df_projets['id'].apply(lambda x: x in projets_a_ajouter)])

In [ ]:
dict_row=df_projets.iloc[0,:].to_dict()
dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x')&(str(v)!='[]'))}
dict_row2

In [81]:
err=[]
for i,row in df_projets.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x')&(str(v)!='[]'))}
    try:
       r=requests.post('http://185.161.45.213/projects/projects', json = dict_row2, headers={"Authorization":Authorization})
       res= r.json()
       if res.get('status')=='ERR':
           err.append(res)
           if res.get('error').get('code')!=422:
               print(err)
               pp.pprint(err)
    except Exception as e:
        pp.pprint(e)

In [ ]:
len(projets_a_ajouter)

Modifications

In [ ]:
nbr_page=int(requests.get('http://185.161.45.213/projects/projects?where={"type":"ANR"}&projection={"id":1,"year":1}&max_results=500'+f"&page={1}", headers={"Authorization":Authorization}).json()['hrefs']['last']['href'].split('page=')[1])
nbr_page

In [ ]:
list_ids=[]
for i in range(1,nbr_page+1):
    print("page",i)
    page=requests.get('http://185.161.45.213/projects/projects?where={%22type%22:%22ANR%22}&projection={%22id%22:1,%22year%22:1}&max_results=500'+f"&page={i}", headers={"Authorization":Authorization}).json()
    for k in range(len(page['data'])):
        list_ids.append(page['data'][k]['year'])

In [ ]:
max(list_ids)

In [ ]:
df_projets = df_projets[df_projets['id'].apply(lambda x: x in list_ids)]

In [ ]:
"""
en principe, il faut que je mette à jour tous les projets 
qui ne sont pas clos avec les titres résumés et persons, mais 
peut etre qu'on peut mettre à jour automatiquement les projets
des 3 dernières années ? 

"""

err=[]
for id in df_projets.iterrows() :
    url = f"http://185.161.45.213/projects/projects/{id}"
    project=requests.get(url, headers={'Authorization': Authorization}).json()
    head = {"Authorization": Authorization, "If-Match": project['etag'], "Content-Type": "application/json"}

    r = requests.patch(url, json = {"year": 2023}, headers=head)
    res= r.json()
    if res.get('status')=='ERR':
        err.append(res)
        if res.get('error').get('code')!=422:
            print(err)
            pp.pprint(err)


# ENVOI DES PARTENAIRES SUR SCANR

In [8]:
df_partenaires=pd.read_json(f"./DATA/{source}/df_partners_id_structures.json")

In [ ]:
df_partenaires.columns

In [10]:
### ATTENTION, vérifier que les projets sirano sont dans des structures françaises
if source=='IRESP':
    df_partenaires[sources[source]['pays']]=df_partenaires.loc[:,sources[source]['ville']].apply(lambda x: x.split('(')[1].replace(')','') if x.find('(')>=0 else 'France')
    df_partenaires.loc[:,sources[source]['ville']]=df_partenaires.loc[:,sources[source]['ville']].apply(lambda x: x.split('(')[0] if x.find('(')>=0 else x)
  
df_partenaires['address']=df_partenaires.apply(lambda row: address(row,sources[source]['pays'],sources[source]['ville'],source), axis=1)
df_partenaires.loc[:,'id_structure']=df_partenaires.loc[:,'id_structure'].apply(lambda x: x[0] if isinstance(x,list) else x )

In [ ]:
df_partenaires

In [ ]:
if source in ['ANSES','SIRANO']:
    df_partenaires['id']=df_partenaires.apply(lambda row: f"{row[sources[source]['code_projet']]}-{row[str(sources[source]['nom_structure'])+'2']}-{row[sources[source]['nom']]}-{row[sources[source]['prenom']]}" , axis=1)
if source =='REG_IDF':
    df_partenaires['id']=df_partenaires.apply(lambda row: f"{row[sources[source]['code_projet']]}-{row[str(sources[source]['nom_structure'])+'2']}-{row['entite_role']}" , axis=1)
df_partenaires['address']=df_partenaires.apply(lambda row: address(row,sources[source]['pays'],sources[source]['ville'],source), axis=1)
df_partenaires=df_partenaires.rename(columns={sources[source]['nom_structure']: 'name', sources[source]['code_projet']: 'project_id', 'id_structure':'participant_id','Projet.Partenaire.Code_Decision_ANR':'id'})
df_partenaires=df_partenaires[['name','id','project_id','participant_id','address']]
df_partenaires['project_type']=source
df_partenaires['participant_id']=df_partenaires.loc[:,'participant_id'].apply(lambda x: str(x[0]).replace('.0','') if isinstance(x,list) else str(x).split(';')[0].replace('.0',''))
df_partenaires=df_partenaires[['id','project_id', 'project_type', 'participant_id', 'name','address']]
df_partenaires['name'] = df_partenaires['name'].astype(str)
df_partenaires

In [ ]:
df_partenaires[df_partenaires.duplicated(subset=['id'])]

In [ ]:
dict_row=df_partenaires.iloc[1,:].to_dict()
dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
dict_row2

In [ ]:
len(df_partenaires)

In [ ]:
err=[]
for i,row in df_partenaires.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    try:
       r=requests.post('http://185.161.45.213/projects/participations', json = dict_row2, headers={"Authorization":Authorization})
       res= r.json()
       if res.get('status')=='ERR':
           print(i)
           err.append(res)
           if res.get('error').get('code')!=422:
               print(err)
               pp.pprint(err)
    except Exception as e:
        pp.pprint(e)

Mise à jour

In [ ]:
nbr_page=int(requests.get('http://185.161.45.213/projects/participations?where={"project_type":"ANSES"}&projection={"id":1}&max_results=500&page=1', headers={"Authorization":Authorization}).json()['hrefs']['last']['href'].split('page=')[1])

list_ids=[]
for i in range(1,nbr_page+1):
    print("page",i)
    page=requests.get('http://185.161.45.213/projects/participations?where={"project_type":"ANSES"}&projection={"id":1}&max_results=500'+f"&page={i}", headers={"Authorization":Authorization}).json()
    for k in range(len(page['data'])):
        print("k",k)
        list_ids.append(page['data'][k]['id'])
    
projets_a_ajouter=[x for x in list(df_partenaires['id'].drop_duplicates()) if x not in list(pd.Series(list_ids).drop_duplicates())]

projets_a_retirer=[x for x in list_ids if x not in list(df_partenaires['id'])]

df_partenaires = df_partenaires[df_partenaires['id'].apply(lambda x: x in projets_a_ajouter)]


In [ ]:
len(projets_a_ajouter)

In [ ]:
print(df_partenaires)

In [ ]:
len(df_partenaires)

In [108]:
err=[]
for i,row in df_partenaires.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    try:
       r=requests.post('http://185.161.45.213/projects/participations', json = dict_row2, headers={"Authorization":Authorization})
       res= r.json()
       if res.get('status')=='ERR':
           print(i)
           err.append(res)
           if res.get('error').get('code')!=422:
               print(err)
               pp.pprint(err)
    except Exception as e:
        pp.pprint(e)

Modifications

In [ ]:
nbr_page=int(requests.get('http://185.161.45.213/projects/participations?where={"project_type":"ANR","participant_id":{"$exists":false}}&projection={"id":1}&max_results=500'+f"&page={1}", headers={"Authorization":Authorization}).json()['hrefs']['last']['href'].split('page=')[1])

list_ids=[]
for i in range(1,nbr_page+1):
    print("page",i)
    page=requests.get('http://185.161.45.213/projects/participations?where={"project_type":"ANR","participant_id":{"$exists":false}}&projection={"id":1}&max_results=500'+f"&page={i}", headers={"Authorization":Authorization}).json()
    for k in range(len(page['data'])):
        print("k",k)
        list_ids.append(page['data'][k]['id'])

df_partenaires = df_partenaires[df_partenaires['id'].apply(lambda x: x in list_ids)]

In [ ]:
len(list_ids)

In [127]:
df_partenaires=df_partenaires[(df_partenaires.participant_id!='x') & (df_partenaires.participant_id!='None')]

In [ ]:
df_partenaires

In [ ]:
err=[]
for i,row in df_partenaires.iterrows() :
    print(i)
    id=row['id']
    url = f"http://185.161.45.213/projects/participations/{id}"
    project=requests.get(url, headers={'Authorization': Authorization}).json()
    head = {"Authorization": Authorization, "If-Match": project['etag'], "Content-Type": "application/json"}

    r = requests.patch(url, json = {"participant_id": row['participant_id']}, headers=head)
    res= r.json()
    if res.get('status')=='ERR':
        err.append(res)
        if res.get('error').get('code')!=422:
            print(err)
            pp.pprint(err)

In [ ]:
pd.Series([x.get('issues').get('id')[25:] for x in err]).drop_duplicates().tolist()

Faire un nouveau fichier avec toutes les structures manquantes

In [ ]:
struct_manq=pd.read_excel('scanr_partenaires_non_identifies.xlsx')

In [19]:
nom1=struct_manq[struct_manq.New=='IRESP'].drop_duplicates(subset='Nom').Nom

In [15]:
iresp_actuel=pd.read_excel('./missing_ids_structures/partenaires_non_identifies_IRESP.xlsx')

In [20]:
nom2=iresp_actuel['Nom_equipe']

In [ ]:
pd.Series(list(nom1)+list(nom2))

In [ ]:
pd.Series(list(nom1)+list(nom2)).drop_duplicates()